<a href="https://colab.research.google.com/github/Last-remote11/Algorithms/blob/main/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98%EC%B2%AB%EA%B1%B8%EC%9D%8C6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###원 핫 벡터로 단어를 표현하면 모든 단어들의 내적이 0이 되어 유사도를 나타낼 수 없게 된다.<br/>
###원 핫 벡터 대신 임베딩은 간단히 말해 단어의 의미를 벡터화하는 것.<br/>
###단어의 유사성이 높으면 내적했을 때 값이 비교적 크게 될듯?

## 모듈 import, 파일 준비

In [ ]:
import torch
import torch.nn as nn
import string
import random
import re
import time, math
import unidecode

!rm -r data
import os 

# 경로 생성과 데이터 다운로드
try:
  os.mkdir("./data")
except:
  pass
!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

In [ ]:
!pip install unidecode

     |████████████████████████████████| 241 kB 26.7 MB/s 


In [ ]:
all_chars = string.printable
n_chars = len(all_chars)
print(all_chars)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	



## 하이퍼파라미터 정의

In [ ]:
num_epochs = 2000
print_every = 100
plot_every = 10

chunk_len = 200

hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [ ]:
# 셰익스피어 예제
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print(file_len)

1115394


모든 파일을 한꺼번에 학습할 수 없다.<br/>
그래서 일정한 크기로 잘라주는데 이것을 앞서 정의했던 chunk 라고 한다.

In [ ]:
# 랜덤한 위치에서 랜덤한 길이의 문자들을 불러옴
def random_chunk():
    # 랜덤한 시작index와 랜덤한 끝index
    start_index = random.randint(0, file_len - chunk_len) 
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

In [ ]:
# 문자들을 앞서 정의했던 문자열 리스트(all_chars)에 해당하는 인덱스로 이루어진 텐서로 변환해주는 함수
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_chars.index(string[c])
    return tensor
    
print(char_tensor('shiranui_flare'))

tensor([28, 17, 18, 27, 10, 23, 30, 18, 88, 15, 21, 10, 27, 14])


In [ ]:
# 앞서 정의한 두 함수를 이용하여 랜덤한 훈련 세트를 만들어주는 함수
def random_train_set():
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target # 입력과 목표

In [ ]:
# RNN 클래스를 만들고 인스턴스화
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        # 임베딩을 70차원으로 한다 하더라도 출력에서는 다시 문자 개수만큼의 벡터로 변환해줘야한다.
        # 그 역할을 디코더가 담당
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.rnn = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        return out,hidden
    
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden

model = RNN(n_chars, embedding_size, hidden_size, n_chars, num_layers)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [ ]:
def test():
    start_str = 'b'
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")

    for i in range(200):
        output, hidden = model(x, hidden)

        output_dist = output.data.view(-1).div(0.8).exp() # 다음에 나올 문자를 확률로 표현한 행렬
        top_i = torch.multinomial(output_dist, 1)[0] 
        # 행렬의 max를 쓰지 않는 이유는 the the the 같이 제일 높은 확률의 문자만 반복되기 때문이다
        predicted_char = all_chars[top_i]

        # 예측한 문자를 출력하고 다음 입력값을 이번 출력값으로 지정
        print(predicted_char,end="")
        x = char_tensor(predicted_char)

## 모델 훈련

In [ ]:
for i in range(num_epochs):
    inp, label = random_train_set()
    hidden = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()

    for j in range(chunk_len - 1):
        x = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor) # 정답
        y, hidden = model(x, hidden)
        loss += loss_func(y, y_) # i번째 에포크의 chunk의 전체 손실을 기록

    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)


 tensor([4.5690], grad_fn=<DivBackward0>) 

TNA%}2T[V(an
gbXl<T*H:IykV"[_A~v~Gp+."prZi-{wz_w&	gAj=*o' h1.EZq=F8v*JY*'u*",[N%E-r'C&cP@
w>';riEbK\j`$gc$1ke7s(*
}*>oM8\

 tensor([2.8615], grad_fn=<DivBackward0>) 

bopu thenet hThe the Wand Ihen ar rrand mes rire?sl be cod Iou te thes hith ses sus shy thd thate I.rg,

emt harl the tou sor leu
Cif tinr sfthes sit!
W hes, thenr.

Hou kerges art or sppor younl

Acpr

 tensor([2.2722], grad_fn=<DivBackward0>) 

bength thy meat I hemld pour goll toit deret too te my theer the my, and rave, homas the saash tht muran to three my the the cod set in the, what it happ ay the the in in tht, in hif at siy ind my thin

 tensor([2.2751], grad_fn=<DivBackward0>) 

beaw ve that be then,
Thate me thencarestongworn ko sertors ufours and porthy the his why the in With wore and me the come not is shan be mangh hhe the her and oen ther for beat?

LUCERETS:
IE:
O stain

 tensor([2.1490], grad_fn=<DivBackward0>) 

boot to this meef mos lones
Los nothern thout